<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Connect-to-workspace" data-toc-modified-id="Connect-to-workspace-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Connect to workspace</a></span></li><li><span><a href="#Define-Datastore" data-toc-modified-id="Define-Datastore-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define Datastore</a></span><ul class="toc-item"><li><span><a href="#Up-load-file-(folder)-to-Datastore" data-toc-modified-id="Up-load-file-(folder)-to-Datastore-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Up load file (folder) to Datastore</a></span></li></ul></li></ul></div>

In [1]:
import azureml.core
from azureml.core import Workspace, Run, Dataset

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment, ScriptRunConfig, Experiment
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

In [3]:
import os

## Connect to workspace

In [4]:
#import workspace from config.json
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.35.0 to work with projet_7


## Define Datastore

In [7]:
#default_ds = ws.get_default_datastore()
#print(default_ds)

{
  "name": "dataset_projet7",
  "container_name": "dataset-projet7",
  "account_name": "stockageprojet7opcr",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


### Up load file (folder) to Datastore

In [11]:
#filepath = os.path.abspath("gtfine")
#filepath

'C:\\Users\\favre\\projet_9\\gtfine'

In azureCLI:

az storage blob upload-batch --destination dataset-projet7 --source C:\\Users\\favre\\projet_9\\sample --account-key ujARUiYsDABZ9UTCzWwzdwaNDscT3dIUsZ/EMvCtx38XhSvZ5Q5Ks5hKn8YnGc5Q+1P9jrMmLR6vl70nQdzrBA==  --account-name stockageprojet7opcr  

## up date env from yml

conda env update --name myenv --file local.yml --prune  
conda env update --name proj9 --file env-p9.yml --prune 
conda env update -p d:\\anaconda\\envs -f env-p9 --prune
  
 --prune uninstalls dependencies which were removed from local.yml

## Compute cluster

In [5]:
cluster_name = 'cluster-projet9'

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', max_nodes=4)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [34]:
AmlCompute.supported_vmsizes(workspace = ws)


[{'name': 'Standard_D1',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 51200},
 {'name': 'Standard_D11',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D11_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D12',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D12_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D13',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 56.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D13_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 56.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D14',
  'vCPUs': 16,
  'gpus': 0,
  'memoryGB': 112.0,
  'maxResourceVolumeMB': 819200},
 {'name': 'Standard_D14_v2',
  'vCPUs': 16,
  'gpus': 0,
  'memoryGB': 112.0,
  'maxResourceVolumeMB': 819200},
 {'name': 'Standard_D15_

## Create environment

In [6]:
#create environment from yml file
env_p9 = Environment.from_conda_specification("env_p9", 'training_azure/env-p9.yml')

No Python version provided, defaulting to "3.6.2"


In [17]:
#env_p9_test = Environment.from_conda_specification("env_p9", 'training_azure/env-p9_test.yml')

No Python version provided, defaulting to "3.6.2"


## Specify docker image 
Base image is CPU and we want to run on GPU

In [7]:
# Specify a GPU base image

env_p9.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
docker_config = DockerConfiguration(use_docker=True)

## Script

In [9]:
%%writefile training_azure/data_augmentation.py
print('print importing lib...')

import argparse
from azureml.core import Run
from azureml.core import Dataset
import shutil
import os
import numpy as np
import pandas as pd
import random
import Augmentor

print("lib imported...")

#get scripts arguments
parser = argparse.ArgumentParser()
parser.add_argument('--image-folder', type=str, dest='image_folder')
parser.add_argument('--mask-folder', type=str, dest='mask_folder')
parser.add_argument('--augmented_data', type=str, dest='augmented_data')
args = parser.parse_args()

save_folder = args.augmented_data

run = Run.get_context()

#load data
print('data...')
mask_path = run.input_datasets['mask']
image_path = run.input_datasets['image']

#
os.makedirs(save_folder, exist_ok=True)

#move picture to intermedieate container for data augmentation
image_dir = image_path
mask_dir = mask_path
mask = '_labelIds'

new_image_dir = os.path.join(save_folder, 'image')
new_mask_dir = os.path.join(save_folder, 'mask')
output_dir = os.path.join(save_folder, 'augmented')
os.makedirs(new_image_dir, exist_ok=True)
os.makedirs(new_mask_dir, exist_ok=True)
reduction = '_leftImg8bit'

#extract list of pictures and rename mask and picture for augmentor ground truth
print('copy file to container')

for root, dirs, files in os.walk(image_dir):
    for name in files:        
        old_path = os.path.join(root, name)
        new_name = name.replace('_leftImg8bit', '')        
        new_path = os.path.join(new_image_dir, new_name)
        if not os.path.exists(new_path):
            shutil.copy(old_path, new_path)
        
        
        


for root, dirs, files in os.walk(mask_dir):
    for name in files:
        if mask in name:
            old_path = os.path.join(root, name)
            new_name = name.replace('_gtFine_labelIds', '')        
            new_path = os.path.join(new_mask_dir, new_name)
            if not os.path.exists(new_path):
                shutil.copy(old_path, new_path)

           

print('build augmentor pipeline')
            
#build augmentor pipeline
p = Augmentor.Pipeline(new_image_dir, output_dir)

#add groud truth for pair modification
p.ground_truth(new_mask_dir)

#add operation
p.rotate(probability=1, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.5)
p.zoom_random(probability=0.5, percentage_area=0.8)
p.skew(probability=0.5, magnitude=0.5)
p.skew_tilt(probability=0.5, magnitude=0.5)
p.random_distortion(probability=0.5,grid_height=4, grid_width=4, magnitude=4)
p.shear(probability=0.5, max_shear_left=10, max_shear_right=10)
p.gaussian_distortion(probability=0.5, corner='bell', method='in', grid_height=4, grid_width=4, magnitude=4)
p.skew_top_bottom(probability=0.5, magnitude=.5)
p.skew_left_right(probability=0.5, magnitude=.5)
p.skew_corner(probability=0.5, magnitude=.5)
p.resize(probability=1,width=256, height=256)

print('start sampling')
p.sample(100)

run.complete()

Overwriting training_azure/data_augmentation.py


In [42]:
%%writefile training_azure/training_kerasHP.py
print('print importing lib...')

import argparse
from azureml.core import Run
from azureml.core import Dataset
import joblib
import os
import numpy as np
import pandas as pd
import random
import json

import tensorflow
from tensorflow.python.keras.utils import Sequence, multi_gpu_model
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.python.keras.optimizers import Adadelta, Nadam
from tensorflow.python.keras.models import Model, load_model
#from tensorflow.python.keras.utils import multi_gpu_model, plot_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping#, ReduceLROnPlateau
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.losses import binary_crossentropy
from tensorboard.plugins.hparams import api as hp

from tensorflow.python.keras.callbacks import Callback

from sklearn.model_selection import train_test_split

from dilatednet import DilatedNet
from multiclassunet import Unet

print('lib imported...')


#get scripts arguments
parser = argparse.ArgumentParser()
parser.add_argument('--augmented-data', type=str, dest='augmented_data')
args = parser.parse_args()

save_folder = args.augmented_data

run = Run.get_context()

print('coef and loss...')

def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss

print('hyper parameters...')

HP_BATCHSIZE = hp.HParam('batch size', hp.Discrete([2, 4]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_LOSS = hp.HParam('loss', hp.Discrete(['categorical_crossentropy', 'dice_loss']))

METRIC_ACCURACY = 'accuracy'

with tensorflow.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_BATCHSIZE, HP_OPTIMIZER, HP_LOSS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

#set parameters
batch_size = hparams[HP_BATCHSIZE]
samples = 50000
steps = samples//batch_size
img_height, img_width = 256, 256
classes = 8
filters_n = 64

#load data
print('data...')
image_dir = os.path.join(save_folder, 'image')
mask_dir = os.path.join(save_folder, 'mask')
augmented_dir = os.path.join(save_folder, 'augmented')


#divison of labelling
cats = {'void': [0, 1, 2, 3, 4, 5, 6],
 'flat': [7, 8, 9, 10],
 'construction': [11, 12, 13, 14, 15, 16],
 'object': [17, 18, 19, 20],
 'nature': [21, 22],
 'sky': [23],
 'human': [24, 25],
 'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]}


#extract list of pictures 
print('making picture list...')
image_list = []
mask_list = []

for root, dirs, files in os.walk(mask_dir):
    for name in files:
        mask_list.append(os.path.join(root, name))
        
for root, dirs, files in os.walk(image_dir):
    for name in files:
        image_list.append(os.path.join(root, name))

for root, dirs, files in os.walk(augmented_dir):
    for name in files:
        if 'groundtruth' in name:
            mask_list.append(os.path.join(root, name))
        else:
            image_list.append(os.path.join(root, name))
        
print("split..")

def train_test_val_split(X, Y, split=(0.2, 0.1), shuffle=True):
    """Split dataset into train/val/test subsets by 70:20:10(default).
    
    Args:
      X: List of data.
      Y: List of labels corresponding to data.
      split: Tuple of split ratio in `test:val` order.
      shuffle: Bool of shuffle or not.
      
    Returns:
      Three dataset in `train:test:val` order.
    """
    
    assert len(X) == len(Y), 'The length of X and Y must be consistent.'
    X_train, X_test_val, Y_train, Y_test_val = train_test_split(X, Y, 
        test_size=(split[0]+split[1]), shuffle=shuffle)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, 
        test_size=split[1], shuffle=False)
    return (X_train, Y_train), (X_test, Y_test), (X_val, Y_val) 

print("class gen...")

class seg_gen(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        #make a array of length batch size containing random position of photo
        idx = np.random.randint(0, 10, batch_size)
        batch_x, batch_y = [], []
        drawn = 0
        
        for i in idx:
            #import original image /255 to normalize and avoid big number for compute power
            _image = image.img_to_array(image.load_img(f'{image_list[i]}', target_size=(img_height, img_width)))/255.
            #import mask
            img = image.img_to_array(image.load_img(f'{mask_list[i]}', grayscale=True, target_size=(img_height, img_width)))
            #assess number of different label
            labels = np.unique(img)
            #ignore picture withtout features
            if len(labels) < 3:
                idx = np.random.randint(0, 50000, batch_size-drawn)
                continue
            img = np.squeeze(img)
            #create a tensor of dimension image with one plan for each category
            mask = np.zeros((img.shape[0], img.shape[1], 8))
            for i in range(-1, 34):
                if i in cats['void']:
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i in cats['flat']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i in cats['construction']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i in cats['object']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i in cats['nature']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i in cats['sky']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                elif i in cats['human']:
                    mask[:,:,6] = np.logical_or(mask[:,:,6],(img==i))
                elif i in cats['vehicle']:
                    mask[:,:,7] = np.logical_or(mask[:,:,7],(img==i))
            mask = np.resize(mask,(img_height*img_width, 8))
            batch_y.append(mask)
            batch_x.append(_image)
            drawn += 1
        return np.array(batch_x), np.array(batch_y)

print('model...')

#with strategy.scope():
unet = DilatedNet(256, 256, 8,use_ctx_module=True, bn=True)
#p_unet = multi_gpu_model(unet, 2) with multi gpu you compile with p_unet
unet.compile(optimizer=hparams['HP_OPTIMIZER'], loss=hparams['HP_LOSS'], metrics=[dice_coeff, 'accuracy'])
    
tb = TensorBoard(log_dir='logs', write_graph=True)
hp = hp.KerasCallback(log_dir='logs', hparams)
mc = ModelCheckpoint(mode='max', filepath='models/pdilated.h5', monitor='acc', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='acc', patience=6, verbose=1)
callbacks = [tb, mc, es, hp]

print('making input...')
train_image, train_mask, val_image, val_mask = train_test_split(image_list, mask_list, test_size=0.15) 

train_gen = seg_gen(train_image[:16], train_mask[:16], batch_size)
valid_gen = seg_gen(val_image[:16], val_mask[:16], batch_size)

print('fit...')
#with multi gpu fit p_unet
unet.fit_generator(train_gen, epochs=5, callbacks=callbacks, validation_data=valid_gen)

print('Saving final weights')
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'dilated.h5')
#save weight from unet (not p_unet)
unet.save_weights(model_file)

run.complete()

Overwriting training_azure/training_hdparams.py


# Pipeline

## Configuration

In [8]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import TensorflowConfiguration

#experiment folder
experiment_folder = 'training_azure'

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

#assign compute
pipeline_run_config.target = training_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = env_p9

#Assign docker image
pipeline_run_config.docker=docker_config

#create distributed config
#distr_config = TensorflowWorkerCount(worker_count=2, parameter_server_count=0)
#distr_config = PipelineParameter(worker_count=2, parameter_server_count=0)

## Step

In [13]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep


# Get the training dataset
image_ds = ws.datasets.get("image_sample")
mask_ds = ws.datasets.get('mask_sample')

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
#with no specification in OutputFileDatasetConfig for destination it will be in workspaceblobstore datastore
augmented_data = OutputFileDatasetConfig("augmented_data")

#arguments data prep
args_prep = ['--image-folder', image_ds.as_named_input('image').as_mount(),
        '--mask-folder', mask_ds.as_named_input('mask').as_mount(),
             '--sample', 2500,
        '--augmented_data', augmented_data]

#argument training
args_training = ['--augmented-data', augmented_data.as_input(),
                '--batch-size', 64,
                '--optimizer', 'adam',
                '--loss', 'categorical_crossentropy',
                '--epoch', 50]

#step1, run the data prep script
prep_step = PythonScriptStep(name = "Data augmentation",
                                source_directory = experiment_folder,
                                script_name = "data_augmentation.py",
                                arguments = args_prep,
                                compute_target = training_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train Model",
                                source_directory = experiment_folder,
                                script_name = "training_param.py",
                                arguments = args_training,
                                compute_target = training_cluster,
                                runconfig = pipeline_run_config,
                                #runconfig_pipeline_params=distr_config,
                                allow_reuse = True)

## Build pipeline

In [14]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)


# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'recommender-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)


pipeline_run.wait_for_completion(show_output=True)

Created step Data augmentation [61f700fb][58538374-e344-457a-8234-e154c018ebe7], (This step will run and generate new outputs)
Created step Train Model [eca2e258][6a0de2ef-6908-46bb-ba1b-48571e5f91e9], (This step will run and generate new outputs)
Submitted PipelineRun 23504de0-0d67-40b6-a34b-523be9e11ddf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/23504de0-0d67-40b6-a34b-523be9e11ddf?wsid=/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourcegroups/Projet_7/workspaces/projet_7&tid=33e47288-d1e1-43e8-b65b-4ba7bfd37a9f
PipelineRunId: 23504de0-0d67-40b6-a34b-523be9e11ddf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/23504de0-0d67-40b6-a34b-523be9e11ddf?wsid=/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourcegroups/Projet_7/workspaces/projet_7&tid=33e47288-d1e1-43e8-b65b-4ba7bfd37a9f
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 5e8c4194-08e1-47a5-be59-e4fbc80c34d6
Link to Azure Machine Learning Portal: https




StepRunId: 86eade7b-931b-43ab-bf43-4ce318b7cd07
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/86eade7b-931b-43ab-bf43-4ce318b7cd07?wsid=/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourcegroups/Projet_7/workspaces/projet_7&tid=33e47288-d1e1-43e8-b65b-4ba7bfd37a9f
StepRun( Train Model ) Status: Running


ClientRequestError: Error occurred in request., ConnectionError: HTTPSConnectionPool(host='westeurope.api.azureml.ms', port=443): Max retries exceeded with url: /pipelines/v1.0/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourceGroups/Projet_7/providers/Microsoft.MachineLearningServices/workspaces/projet_7/PipelineRuns/23504de0-0d67-40b6-a34b-523be9e11ddf/Graph/Node/StatusCode (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000169600205C8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## Examine 

see projet 8

## Script testing

In [15]:
experiment_name = 'image-segmentation-unet-v2'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [16]:
# Get the training dataset
image_ds = ws.datasets.get("image_sample")
mask_ds = ws.datasets.get('mask_sample')

In [17]:
from azureml.core import ScriptRunConfig

args = ['--image-folder', image_ds.as_named_input('image').as_mount(),
        '--mask-folder', mask_ds.as_named_input('mask').as_mount()]

src = ScriptRunConfig(source_directory="training_azure",
                      script='data_augmentation.py', 
                      arguments=args,
                      compute_target=training_cluster,
                      environment=env_p9,
                      docker_runtime_config=docker_config)

In [18]:
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: image-segmentation-unet-v2_1636659097_8f86bedd
Web View: https://ml.azure.com/runs/image-segmentation-unet-v2_1636659097_8f86bedd?wsid=/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourcegroups/Projet_7/workspaces/projet_7&tid=33e47288-d1e1-43e8-b65b-4ba7bfd37a9f

Streaming azureml-logs/20_image_build_log.txt

2021/11/11 19:31:46 Downloading source code...
2021/11/11 19:31:47 Finished downloading source code
2021/11/11 19:31:47 Creating Docker network: acb_default_network, driver: 'bridge'
2021/11/11 19:31:47 Successfully set up Docker network: acb_default_network
2021/11/11 19:31:47 Setting up Docker configuration...
2021/11/11 19:31:48 Successfully set up Docker configuration
2021/11/11 19:31:48 Logging in to registry: 38d02f2005824c85859d3a5c11c9a143.azurecr.io
2021/11/11 19:31:49 Successfully logged into 38d02f2005824c85859d3a5c11c9a143.azurecr.io
2021/11/11 19:31:49 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_netwo

Installing pip dependencies: ...working... 
Ran pip subprocess with arguments:
['/azureml-envs/azureml_070ee6abbff3d00e9dd3351859dea785/bin/python', '-m', 'pip', 'install', '-U', '-r', '/azureml-environment-setup/condaenv.egtf_b2z.requirements.txt']
Pip subprocess output:
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3925 sha256=a8d0337f3a943499715fcf51d3fd3d6c9760b254b838fd69748390975f36f806
  Stored in directory: /root/.cache/pip/wheels/e2/1d

Removing intermediate container f60ffb121c93
 ---> 098db6aaaf04
Step 9/19 : ENV PATH /azureml-envs/azureml_070ee6abbff3d00e9dd3351859dea785/bin:$PATH
 ---> Running in 39c61435284d
Removing intermediate container 39c61435284d
 ---> 7a3f90b69987
Step 10/19 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> ce57c7230b65
Step 11/19 : RUN echo "Copying environment context"
 ---> Running in b57a29bb84a3
Copying environment context
Removing intermediate container b57a29bb84a3
 ---> 6fbca93ba679
Step 12/19 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> d98a0e834212
Step 13/19 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_070ee6abbff3d00e9dd3351859dea785
 ---> Running in 4e3c505fad3b
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with mate


Streaming azureml-logs/65_job_prep-tvmps_89f110d5b4a185db13ced95234445587040446cb197bbd16b9834c58eda55112_d.txt

[2021-11-11T19:45:51.002200] Entering job preparation.
[2021-11-11T19:45:51.835843] Starting job preparation.
[2021-11-11T19:45:51.835877] Extracting the control code.
[2021-11-11T19:45:51.836173] Starting extract_project.
[2021-11-11T19:45:51.836215] Starting to extract zip file.
[2021-11-11T19:45:51.883546] Finished extracting zip file.
[2021-11-11T19:45:51.886531] Using urllib.request Python 3.0 or later
[2021-11-11T19:45:51.886569] Start fetching snapshots.
[2021-11-11T19:45:51.886599] Start fetching snapshot.
[2021-11-11T19:45:51.886614] Retrieving project from snapshot: 4b07b895-bc69-438e-a1af-daaf08e9b86f
Starting the daemon thread to refresh tokens in background for process with pid = 54
[2021-11-11T19:45:53.036678] Finished fetching snapshot.
[2021-11-11T19:45:53.036718] Finished fetching snapshots.
[2021-11-11T19:45:53.036729] Finished extract_project.
[2021-11-11

{'runId': 'image-segmentation-unet-v2_1636659097_8f86bedd',
 'target': 'cluster-projet9',
 'status': 'Completed',
 'startTimeUtc': '2021-11-11T19:45:26.018744Z',
 'endTimeUtc': '2021-11-11T19:47:27.568302Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '4b07b895-bc69-438e-a1af-daaf08e9b86f',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '040b6427-bb4e-4070-a625-015eed30351e'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'image', 'mechanism': 'Mount'}}, {'dataset': {'id': 'a3ec240d-fd00-4be5-94ce-d895cbefb532'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'mask', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'data_augmentation.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--image-folder',
   'DatasetConsumptionConfig:image',
   '--mask-folder',
   'Data

In [74]:
for i in range(0,10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [38]:
from tensorflow.keras.utils import Sequence